In [1]:
#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import nltk
import re
import string

### Features for classifiers
(&)in data (+) calculate

#### Query Content:  
&Date of creation of post-User account creation date  
+Count of lowercase chars in body  
+Count of uppercase chars in body  
+lowercase to uppercase ratio in body  
+character count in body  (removed)  
+word count in body  
+code length in body  
+length of first line in body   
+punctuations count  
+sentence count in body  
+count of sentences starting with you  
+count of sentences starting with i  
+count of interrogative sentences  
+count of URLs in body  
+count of stackoverflow URLs in body  
+count of short words in body  
+count of tags  
+tags weight(to be worked upon)  

+character count in title  
+word count in title     
+title starting with what/which/how (useful for opinion based questions) or use interrogative words check


#### User profile

&Reputation  
&View count  
&upvotes received  
&downvotes received   
&about me filled  
&website url filled  
&location filled  
&profile image url filled    
questions asked by user  
answers posted by user  
CloseVotesReceived: The number of close votes received by the user for his posts  
Previous Questions with -ve score  
Previous Questions with +ve score  
Previous Questions with 0 score  
Previous Answers with -ve score  
Previous Answers with +ve score  
Previous Answers with 0 score  

##### URL count
https://codeblog.jonskeet.uk/2010/08/29/writing-the-perfect-question/
Often there will be other people in a similar situation, but the answers didn’t quite match your situation. Just like the above point about unusual constraints, it saves time if you can point out differences between your situation and other common ones. It’s even worth referring to other related questions explicitly – particularly if they’re on the same forum. Aside from anything else, this shows a certain amount of “due diligence” – people are generally more willing to help you if can show you’ve already put some effort in.  

Sharing your research helps everyone. Tell us what you found and why it didn’t meet your needs. This demonstrates that you’ve taken the time to try to help yourself, it saves us from reiterating obvious answers, and above all, it helps you get a more specific and relevant answer!

##### Title length
Write a title that summarizes the specific problem
At the same time a “question” title of “Please help” is unlikely to do well

#### Count of interrogative words
Make sure it’s obvious what you’re trying to get out of the question. Too many “questions” are actually just statements: when I do X, something goes wrong. Well, what did you expect it to do? What are you trying to accomplish? What have you already tried? What happened in those attempts?

#### code length
Shows effort done on the part of asker.  
BUT  
Be as short as possible. If I have to wade through hundreds of lines of code to find the problem, I’m doing work that you should be doing


#### length of first sentence
There’s no need to include greetings and sign-offs such as “Hi everyone!” and “Thanks – hope to get an answer soon” in the question. These will often be edited out by other users, as they’re basically a distraction. Greetings at the start of a question are particularly useless as they can take up valuable space in the snippet displayed in the question list.

##### some closing weight for tags such as homework?

In [2]:
#Resources for text cleaning: 
#https://machinelearningmastery.com/clean-text-machine-learning-python/
#https://github.com/SudalaiRajkumar/NLP/blob/master/src/Text%20Cleaning.ipynb
#https://docs.python.org/3/library/re.html

#Returns number of lowercase chars in the text
def lowercaseCount(text):
    lowercount=0
    try:
        for w in text:
            for char in w:
                if(char.islower()):
                    lowercount+=1
        return lowercount
    except:
        #print(type(text))
        print(text)

#Returns number of uppercase chars in the text
def uppercaseCount(text):
    uppercount=0
    try:    
        for w in text:
            for char in w:
                if(char.isupper()):
                    uppercount+=1
        return uppercount
    except:
        print(text)
    
#Returns count of total number of words in the query
#https://stackoverflow.com/questions/19410018/how-to-count-the-number-of-words-in-a-sentence-ignoring-numbers-punctuation-an
def countWords(text):
    return len(text.split())

#Removes HTML from query data but doesn't remove the content bw opening and closing tag
#Attributes are removed eg:<a href="this is all removed">not removed</a>
#https://stackoverflow.com/questions/3398852/using-python-remove-html-tags-formatting-from-a-string
def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)
    
#Sentence tokenization
from nltk import sent_tokenize
#Tokenizes sentences into sentence list
def getSentTokens(sentences):
    return sent_tokenize(sentences)

#Count of sentences in the post body
def getSentenceCount(text):
    return len(split_into_sentences(text))

def getFirstLineLength(text):
    sentences=split_into_sentences(text)
    #print(len(sentences))
    if(len(sentences)==0):
        #print(text)
        return 0
    firstLine=sentences[0]
    return len(firstLine)
    
def countOfInterrogativeSent(text):
    text=text.lower()
    #tokens=text.split() #1D list of words 
    interrogatives=re.findall(r'\b[a-z]*[^.!][?]',text)
    return len(interrogatives)
    
#Word tokenization
from nltk import word_tokenize
#Tokenizes sentence 1D list into words 2D list
def getWordTokens(sentenceList):
    return [word_tokenize(s) for s in sentenceList]

#Punctuations removal
def removePunctuations(word):
    return re.sub(r'\W+', '', word)

#Stop words removal from list of words
from nltk.corpus import stopwords
eng_stop=set(stopwords.words('english'))
#print(eng_stop)
def removeStopwords(text):
    return [word for word in text if word not in eng_stop]

def sentencesStartWithYouCount(text):
    count=0
    text=text.lower()
    text=re.sub('<.*>','',text)
    sentences=split_into_sentences(text)
    for sentence in sentences:
        tokens=re.split('[\s;\',.\-\%]',sentence)
        #print(tokens)
        if(tokens[0]=="you"):
            #print("token0="+tokens[0])
            count+=1
    return count;

def sentencesStartWithICount(text):
    count=0
    text=text.lower()
    sentences=split_into_sentences(text)
    for sentence in sentences:
        tokens=re.split('[\s;\',.\-\%]',sentence)
        #print(tokens)
        if(tokens[0]=="i"):
            #print("token0="+tokens[0])
            count+=1
    return count;

#https://stackoverflow.com/questions/6883049/regex-to-extract-urls-from-href-attribute-in-html-with-python
#https://stackoverflow.com/questions/1374457/find-out-how-many-times-a-regex-matches-in-a-string-in-python
#Returns count of url in a given post
def urlCount(text):
    urls=re.findall(r'https://?|ftp://',text)
    #print(urls)
    return len(urls)

#Returns count of URls pointing to Stack Overflow 
def SOUrlCount(text):
    SOUrls=re.findall(r'https://stackoverflow.com',text)
    #print(SOUrls)
    return len(SOUrls)
    
#Returns the length of total code present in the post
def codeLength(text):
    #make sure you don't strip HTML beforehand
    codes=re.findall(r"<code>(.*?)</code>",text,flags= re.DOTALL)
    #print(codes)
    return len(''.join(codes))

def makeBinary(text):
    if(pd.isnull(text)):
        return 0
    return 1

#https://stackoverflow.com/questions/12628958/remove-small-words-using-python
def countShortWords(text):
    text=striphtml(text)
    shortwords = re.findall(r'\W*\b\w{1,3}\b',text)
    #print(shortwords)
    return len(shortwords)

#Returns the count of tags in the post
#https://stackoverflow.com/questions/11301387/python-regex-first-shortest-match
def countTagsForClosed(text):
    tagList=re.findall('<.*?>',text)
    #print(tagList)
    return len(tagList)

def countTagsForOpen(text):
    tagList=text.split('|')
    #print(tagList)
    return len(tagList)

,

In [3]:
#Credits: https://stackoverflow.com/questions/4576077/python-split-text-on-sentences

alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
digits = "([0-9])" 

def split_into_sentences(text):
    #text=body
    text=striphtml(text)
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    #print(text)
    return sentences

In [64]:
# text = ("This's is@ 9 short, sentence. Amazing! I have tried an approach. I am trying this. I'm trying. " 
#        "You should suggest some, too. You've done this before?")
# text=text.lower()
# print(split_into_sentences(text))
# print("Lowercase count:"+str(lowercaseCount(text)))
# print("Uppercase count:"+str(uppercaseCount(text)))
# #print("Lowercase chars to Upper chars ratio:"+str( lowercaseCount(text)/uppercaseCount(text) ))
# text=text.lower()
# print(text);

In [65]:
# text2=("<p>Are there any free web based service to manage/use your remote computers?</p> "
# "<p>I use <a href=\"https://secure.logmein.com/home.asp\" rel=\"nofollow noreferrer\">Logmein</a> free. "
# "It solves the purpose. But wondering any other free services available.</p> "
# "<p>Heard <a href=\"https://www.copilot.com/\" rel=\"nofollow noreferrer\">Copilot</a> is great but not free!"
#        "<a href=\"https://stackoverflow.com/questions/51884419/how-to-paginate-stack-exchange-data-explorer-sede-results\" "
#       )
# print(urlCount(text2))
# print(SOUrlCount(text2))

In [66]:
# text3=("some text <code> something here </code> some text <code>hello"
# "world</code>")
# print(codeLength(text3))

In [67]:
# sentences=getSentTokens(text)
# print("Sentence count:"+str(len(sentences)))
# print(sentences)

In [68]:
# tokens=getWordTokens(sentences)
# #tokens is 2D list
# for sentence in tokens:
#     print(sentence)

In [69]:
# print(text)
# print("Number of sentences starting with you:"+str(sentencesStartWithYouCount(text)) )
# print("Number of sentences starting with i:"+str(sentencesStartWithICount(text)) ) 

In [70]:
# tokens=[removePunctuations(w) for s in tokens for w in s]
# tokens=list( filter(None,tokens) )
# print(tokens)
# print(len(tokens))

In [71]:
# tokens=removeStopwords(tokens)
# print(tokens)
# print(len(tokens))

In [72]:
# print(string.punctuation)

If HistoryClosedDate is not null and ClosedDate is null
then the post might be protected
PostId=151969

Multiple rows for same PostId=101754 in Off topic closed question database

In [73]:
# data=pd.read_csv("OffTopic1.csv")
# print(data.head())

In [74]:
# #Adding necessary feature columns
# #https://stackoverflow.com/questions/40045632/adding-a-column-in-pandas-df-using-a-function

# data['lowerUpperRatio']=data['Body'].apply(lowercaseCount)/data['Body'].apply(uppercaseCount)
# data['sentenceCount']=data['Body'].apply(getSentenceCount)
# data['firstLineLength']=data['Body'].apply(getFirstLineLength)
# data['codeLength']=data['Body'].apply(codeLength)
# data['urlCount']=data['Body'].apply(urlCount)
# data['SOUrlCount']=data['Body'].apply(SOUrlCount)
# data['titleLengthInChars']=data['Title'].apply(len)
# data['countOfInterrogativeSent']=data['Body'].apply(countOfInterrogativeSent)
# data['sentencesStartWithYouCount']=data['Body'].apply(sentencesStartWithYouCount)
# data['sentencesStartWithICount']=data['Body'].apply(sentencesStartWithICount)
# data['shortWordCount']=data['Body'].apply(countShortWords)
# data['bodyWordCount']=data['Body'].apply(countWords)
# data['tagCount']=data['Tags'].apply(countTags)
# data['punctuationCount']=data['Body'].apply(countPunctuations)

# data['websiteUrlFilled']=data['WebsiteUrl'].apply(makeBinary)
# data['locationFilled']=data['Location'].apply(makeBinary)
# data['aboutMeFilled']=data['AboutMe'].apply(makeBinary)
# data['profileImageUrlFilled']=data['ProfileImageUrl'].apply(makeBinary)

# print(data.head())

In [75]:
# data.columns

In [76]:
# removeTags=data.iloc[2].Body
# print(data.iloc[2].Body)
# removeTags=striphtml(removeTags)
# print(removeTags)
# print(countShortWords(removeTags))
# print(split_into_sentences(removeTags))

In [77]:
# text5="I am Ph.D and A.B.c.de. He said, \"Hello!\""
# print((text5))
# text5 = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text5)
# if "Ph.D" in text5: text5 = text5.replace("Ph.D.","Ph<prd>D<prd>")
# print((text5))
# if "”" in text5: text5 = text5.replace(".”","”.")
# print((text5))
# if "\"" in text5: text5 = text5.replace(".\"","\".")
# if "!" in text5: text5 = text5.replace("!\"","\"!")
# print((text5))
# if "?" in text5: text5 = text5.replace("?\"","\"?")
# text5 = text5.replace(".",".<stop>")
# text5 = text5.replace("?","?<stop>")
# text5 = text5.replace("!","!<stop>")
# text5 = text5.replace("<prd>",".")
# print(text5)
# sentences = text5.split("<stop>")
# print(sentences)

In [78]:
# text6=("Hello everyone! \n"
# "I want to ask a question.")
# print(text6)
# print(getFirstLineLength(text6))
# print(text6)

In [79]:
# re.findall(r'[!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~]',"hello! he\ @")

In [21]:
# print(countTags(data.iloc[2].Tags))

2


In [81]:
# punc=data.iloc[11].Body
# print(punc)
# print( countPunctuations(punc) )

In [4]:
#READING CSV Files
offTopic=pd.read_csv("102_OffTopic.csv")
unclear=pd.read_csv("103_Unclear.csv")
tooBroad=pd.read_csv("104_TooBroad.csv")
opinionBased=pd.read_csv("105_OpinionBased.csv")
openQues=pd.read_csv("open.csv")
#https://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options
print(offTopic.head())

C:\Users\Lenovo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,1,3,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Lenovo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


   PostId UserId        ClosedDate Comment  \
0  106963      4   21/03/2018 7:42     102   
1   33242      5  12/08/2017 17:30     102   
2   53802     20  19/02/2014 14:01     102   
3      72     25   20/04/2017 7:24     102   
4   15731     26   16/03/2015 1:49     102   

                                                Body  \
0  <p>I'm building the world's simplest library a...   
1  <p>Is there method (other than trial and error...   
2  <p>I'm currently working on a JavaScript tool ...   
3  <p>I want to format my existing comments as 'R...   
4  <p>I am looking for either a FireFox extension...   

                                               Title  \
0  How can I lookup data about a book from its ba...   
1  How can I find unused images and CSS styles in...   
2  What is the best tool to benchmark my JavaScript?   
3    How do I add existing comments to RDoc in Ruby?   
4  Looking for a specific FireFox extension / pro...   

                                        Tags Repu

In [5]:
print("offTopic before sampling: "+str(offTopic.shape))
print("unclear before sampling: "+str(unclear.shape))
print("tooBroad before sampling: "+str(tooBroad.shape))
print("opinionBased before sampling: "+str(opinionBased.shape))
print("open questions data before sampling: "+str(openQues.shape))

#Cleaning data
#Dropping those examples where ClosedDate=NAN(signifies the question wasn't actually closed but locked, protected)
#https://stackoverflow.com/questions/36370839/better-way-to-drop-nan-rows-in-pandas?
print("\n<<< removing rows with NaN >>>\n")
offTopic.dropna(subset=['ClosedDate','Body','Title'],inplace = True) 
unclear.dropna(subset=['ClosedDate','Body','Title'],inplace = True) 
tooBroad.dropna(subset=['ClosedDate','Body','Title'],inplace = True) 
opinionBased.dropna(subset=['ClosedDate','Body','Title'],inplace = True) 
openQues.dropna(subset=['Body','Title'],inplace = True) 

print("cleaned offTopic before sampling: "+str(offTopic.shape))
print("cleaned unclear before sampling: "+str(unclear.shape))
print("cleaned tooBroad before sampling: "+str(tooBroad.shape))
print("cleaned opinionBased before sampling: "+str(opinionBased.shape))
print("cleaned open questions data before sampling: "+str(openQues.shape))

offTopic before sampling: (125000, 16)
unclear before sampling: (53821, 16)
tooBroad before sampling: (53374, 16)
opinionBased before sampling: (22808, 16)
open questions data before sampling: (112860, 14)

<<< removing rows with NaN >>>

cleaned offTopic before sampling: (117067, 16)
cleaned unclear before sampling: (48803, 16)
cleaned tooBroad before sampling: (50440, 16)
cleaned opinionBased before sampling: (22018, 16)
cleaned open questions data before sampling: (112860, 14)


In [6]:
#Sampling 20k rows from each close reason type
#https://stackoverflow.com/questions/15923826/random-row-selection-in-pandas-dataframe

#Every run will generate new random samples
offTopicSample=offTopic.sample(n=20000)  #n=20k 
unclearSample=unclear.sample(n=20000)  #n=20k 
tooBroadSample=tooBroad.sample(n=20000)  #n=20k 
opinionBasedSample=opinionBased.sample(n=20000)  #n=20k 
openQuesSample=openQues.sample(n=80000) #n=80k (sum total of all closed questions)

#print(offTopicSample.head())
#print( unclearSample.head() )
#print( tooBroadSample.head() )
#print( opinionBasedSample.head() )
#print( openQuesSample.head() )

In [51]:
offTopicSample.to_csv('offTopicSample.csv')
unclearSample.to_csv('unclearSample.csv')
tooBroadSample.to_csv('tooBroadSample.csv')
opinionBasedSample.to_csv('opinionBasedSample.csv')
openQuesSample.to_csv('openQuesSample.csv')

In [52]:
offTopicSample=pd.read_csv("offTopicSample.csv")
unclearSample=pd.read_csv("unclearSample.csv")
tooBroadSample=pd.read_csv("tooBroadSample.csv")
opinionBasedSample=pd.read_csv("opinionBasedSample.csv")
openQuesSample=pd.read_csv("openQuesSample.csv")

### CLASSIFIER FOR OPEN/CLOSED QUESTIONS

In [7]:
print("offTopicSample: "+str(offTopicSample.shape))
print("unclearSample: "+str(unclearSample.shape))
print("tooBroadSample: "+str(tooBroadSample.shape))
print("opinionBasedSample: "+str(opinionBasedSample.shape))
print("openQuesSample: "+str(openQuesSample.shape))

offTopicSample: (20000, 16)
unclearSample: (20000, 16)
tooBroadSample: (20000, 16)
opinionBasedSample: (20000, 16)
openQuesSample: (80000, 14)


In [9]:
#Merging dataframes into a single one
closedQuesSample=pd.concat([offTopicSample,unclearSample,tooBroadSample,opinionBasedSample])

print("closedQuesSample: "+str(closedQuesSample.shape))

closedQuesSample: (80000, 16)


In [10]:
# closedQuesSample.columns

Index(['PostId', 'UserId', 'ClosedDate', 'Comment', 'Body', 'Title', 'Tags',
       'Reputation', 'experienceInTime', 'WebsiteUrl', 'Location', 'AboutMe',
       'Views', 'Upvotes', 'Downvotes', 'ProfileImageUrl'],
      dtype='object')

In [11]:
# openQuesSample.columns

Index(['PostId', 'UserId', 'Body', 'Title', 'Tags', 'Reputation',
       'experienceInTime', 'WebsiteUrl', 'Location', 'AboutMe', 'Views',
       'Upvotes', 'Downvotes', 'ProfileImageUrl'],
      dtype='object')

In [13]:
closedQuesSample['tagCount']=closedQuesSample['Tags'].apply(countTagsForClosed)
openQuesSample['tagCount']=openQuesSample['Tags'].apply(countTagsForOpen)

In [14]:
closedQuesSample.columns

Index(['PostId', 'UserId', 'ClosedDate', 'Comment', 'Body', 'Title', 'Tags',
       'Reputation', 'experienceInTime', 'WebsiteUrl', 'Location', 'AboutMe',
       'Views', 'Upvotes', 'Downvotes', 'ProfileImageUrl', 'tagCount'],
      dtype='object')

In [15]:
openQuesSample.columns

Index(['PostId', 'UserId', 'Body', 'Title', 'Tags', 'Reputation',
       'experienceInTime', 'WebsiteUrl', 'Location', 'AboutMe', 'Views',
       'Upvotes', 'Downvotes', 'ProfileImageUrl', 'tagCount'],
      dtype='object')

In [12]:
# tagsx="android|c++|android-ndk|build.gradle"
# print(countTagsForOpen(tagsx))

In [16]:
#Adding a column for classifying between open and closed data
closedQuesSample['closed']=1
openQuesSample['closed']=0

# print(closedQuesSample.columns)
# print(openQuesSample.columns)

Index(['PostId', 'UserId', 'ClosedDate', 'Comment', 'Body', 'Title', 'Tags',
       'Reputation', 'experienceInTime', 'WebsiteUrl', 'Location', 'AboutMe',
       'Views', 'Upvotes', 'Downvotes', 'ProfileImageUrl', 'tagCount',
       'closed'],
      dtype='object')
Index(['PostId', 'UserId', 'Body', 'Title', 'Tags', 'Reputation',
       'experienceInTime', 'WebsiteUrl', 'Location', 'AboutMe', 'Views',
       'Upvotes', 'Downvotes', 'ProfileImageUrl', 'tagCount', 'closed'],
      dtype='object')


In [20]:
#Removing unnecessary columns(PostId, UserId, ClosedDate etc) and streamlining all data for one dataframe
closedQuesSample.drop(['ClosedDate'],inplace=True,axis=1)

#Adding a 'Comment' column in Open Questions with value=100 
openQuesSample['Comment']=100

print(closedQuesSample.shape)
print(openQuesSample.shape)

(80000, 17)
(80000, 17)


In [26]:
openQuesSample.head()

,PostId,UserId,Comment,Body,Title,Tags,Reputation,experienceInTime,WebsiteUrl,Location,AboutMe,Views,Upvotes,Downvotes,ProfileImageUrl,tagCount,closed
8287,32203002,5218650,100,<p>Please someone explain complete step by ste...,How to build a kernel in Minix 3?,kernel|minix,6,13,NaN,NaN,NaN,8,0,0,https://lh4.googleusercontent.com/-_GtduZ_HroM...,2,0
34655,22080350,1375553,100,<p>The following JSON is returned from SharePo...,ConvertFrom-Json : Cannot convert the JSON str...,json|rest|powershell,33857,664,http://blog.vgrem.com/,"Helsinki, Finland",<p>Currently a software developer at HiQ Finla...,2438,1574,2413,https://i.stack.imgur.com/w2whQ.png,3,0
1593,28019349,1774679,100,<p>What's the benifit of allocating a chunk of...,Is it faster when access the contiguous physic...,linux|memory-management|linux-kernel|memory-ad...,1470,816,NaN,中国Shanghai Shi,<p>Love the computer science &amp; technology ...,312,362,3,https://www.gravatar.com/avatar/a33ac21cad99e5...,5,0
76697,32211159,411954,100,<p>I'm using an S3 bucket as the origin or a C...,Possible to force CloudFront to sync with enti...,amazon-web-services|amazon-s3|cdn|amazon-cloud...,10992,1846,NaN,NaN,NaN,667,269,77,NaN,4,0
91724,29798226,4819459,100,<p>I am making a chat app with AngularJs and I...,Add class to newest items that are added into ...,javascript|arrays|angularjs,34,0,NaN,Somewhere,NaN,15,8,0,https://www.gravatar.com/avatar/3e58774529ac97...,3,0


In [27]:
print(closedQuesSample.columns)
print(openQuesSample.columns)

#Rearranging openQuesSample dataframe columns
#https://stackoverflow.com/questions/13148429/how-to-change-the-order-of-dataframe-columns
cols = list(openQuesSample.columns.values)
print(cols)
cols=['PostId', 'UserId', 'Comment', 'Body', 'Title', 'Tags', 'Reputation', 'experienceInTime', 'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'Upvotes', 'Downvotes', 'ProfileImageUrl', 'tagCount', 'closed']
openQuesSample=openQuesSample[cols]
print(closedQuesSample.columns)
print(openQuesSample.columns)

Index(['PostId', 'UserId', 'Comment', 'Body', 'Title', 'Tags', 'Reputation',
       'experienceInTime', 'WebsiteUrl', 'Location', 'AboutMe', 'Views',
       'Upvotes', 'Downvotes', 'ProfileImageUrl', 'tagCount', 'closed'],
      dtype='object')
Index(['PostId', 'UserId', 'Comment', 'Body', 'Title', 'Tags', 'Reputation',
       'experienceInTime', 'WebsiteUrl', 'Location', 'AboutMe', 'Views',
       'Upvotes', 'Downvotes', 'ProfileImageUrl', 'tagCount', 'closed'],
      dtype='object')
['PostId', 'UserId', 'Comment', 'Body', 'Title', 'Tags', 'Reputation', 'experienceInTime', 'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'Upvotes', 'Downvotes', 'ProfileImageUrl', 'tagCount', 'closed']
Index(['PostId', 'UserId', 'Comment', 'Body', 'Title', 'Tags', 'Reputation',
       'experienceInTime', 'WebsiteUrl', 'Location', 'AboutMe', 'Views',
       'Upvotes', 'Downvotes', 'ProfileImageUrl', 'tagCount', 'closed'],
      dtype='object')
Index(['PostId', 'UserId', 'Comment', 'Body', 'Title', 'Tags', 

In [28]:
openQuesSample.head()

,PostId,UserId,Comment,Body,Title,Tags,Reputation,experienceInTime,WebsiteUrl,Location,AboutMe,Views,Upvotes,Downvotes,ProfileImageUrl,tagCount,closed
8287,32203002,5218650,100,<p>Please someone explain complete step by ste...,How to build a kernel in Minix 3?,kernel|minix,6,13,NaN,NaN,NaN,8,0,0,https://lh4.googleusercontent.com/-_GtduZ_HroM...,2,0
34655,22080350,1375553,100,<p>The following JSON is returned from SharePo...,ConvertFrom-Json : Cannot convert the JSON str...,json|rest|powershell,33857,664,http://blog.vgrem.com/,"Helsinki, Finland",<p>Currently a software developer at HiQ Finla...,2438,1574,2413,https://i.stack.imgur.com/w2whQ.png,3,0
1593,28019349,1774679,100,<p>What's the benifit of allocating a chunk of...,Is it faster when access the contiguous physic...,linux|memory-management|linux-kernel|memory-ad...,1470,816,NaN,中国Shanghai Shi,<p>Love the computer science &amp; technology ...,312,362,3,https://www.gravatar.com/avatar/a33ac21cad99e5...,5,0
76697,32211159,411954,100,<p>I'm using an S3 bucket as the origin or a C...,Possible to force CloudFront to sync with enti...,amazon-web-services|amazon-s3|cdn|amazon-cloud...,10992,1846,NaN,NaN,NaN,667,269,77,NaN,4,0
91724,29798226,4819459,100,<p>I am making a chat app with AngularJs and I...,Add class to newest items that are added into ...,javascript|arrays|angularjs,34,0,NaN,Somewhere,NaN,15,8,0,https://www.gravatar.com/avatar/3e58774529ac97...,3,0


In [29]:
#Merging open and closed questions
openCloseData=pd.concat([closedQuesSample,openQuesSample])
print("openCloseData: "+str(openCloseData.shape))

openCloseData: (160000, 17)


In [30]:
#Exporting dataframe before adding columns, Now: 17 columns
openCloseData.to_csv('openCloseData.csv')

In [31]:
#Adding necessary feature cJolumns
#https://stackoverflow.com/questions/40045632/adding-a-column-in-pandas-df-using-a-function

openCloseData['lowerUpperRatio']=(openCloseData['Body'].apply(lowercaseCount))/(openCloseData['Body'].apply(uppercaseCount))

In [32]:
openCloseData['sentenceCount']=openCloseData['Body'].apply(getSentenceCount)

In [33]:
openCloseData['firstLineLength']=openCloseData['Body'].apply(getFirstLineLength)

In [34]:
openCloseData['codeLength']=openCloseData['Body'].apply(codeLength)

In [35]:
openCloseData['urlCount']=openCloseData['Body'].apply(urlCount)

In [36]:
openCloseData['SOUrlCount']=openCloseData['Body'].apply(SOUrlCount)

In [37]:
openCloseData['titleLengthInChars']=openCloseData['Title'].apply(len)

In [38]:
openCloseData['countOfInterrogativeSent']=openCloseData['Body'].apply(countOfInterrogativeSent)

In [39]:
openCloseData['sentencesStartWithYouCount']=openCloseData['Body'].apply(sentencesStartWithYouCount)
openCloseData['sentencesStartWithICount']=openCloseData['Body'].apply(sentencesStartWithICount)

In [40]:
openCloseData['shortWordCount']=openCloseData['Body'].apply(countShortWords)
openCloseData['bodyWordCount']=openCloseData['Body'].apply(countWords)
#openCloseData['tagCount']=openCloseData['Tags'].apply(countTags) #Done already 
openCloseData['punctuationCount']=openCloseData['Body'].apply(countPunctuations)

In [41]:
openCloseData['websiteUrlFilled']=openCloseData['WebsiteUrl'].apply(makeBinary)
openCloseData['locationFilled']=openCloseData['Location'].apply(makeBinary) 
openCloseData['aboutMeFilled']=openCloseData['AboutMe'].apply(makeBinary)
openCloseData['profileImageUrlFilled']=openCloseData['ProfileImageUrl'].apply(makeBinary)

print(openCloseData.head())

         PostId   UserId Comment  \
948    21170255     5921     102   
36870  38290647  1601251     102   
3788    6496114    52924     102   
42882  25982771  1968462     102   
14338  10142243   383479     102   

                                                    Body  \
948    <p>I tried to Google for this, but was (surpri...   
36870  <p>I am at a decision fork where i have to pic...   
3788   <p>Does anyone know of a good tool to test Sch...   
42882  <p>The <a href="http://www.postgresql.org/docs...   
14338  <p>Which frameworks would you recommend for wr...   

                                                   Title  \
948    Can anyone point me to (or post) an array of e...   
36870         RESTful API with either Tomcat or Node.js?   
3788                  Tool for testing Schema.org markup   
42882               Inserting into simple Postgres views   
14338  Frameworks for Unittests for Multithreading in...   

                                                    Tags Repu

In [42]:
print("openCloseData shape after adding features:"+str(openCloseData.shape))
print(openCloseData.columns)

openCloseData shape after adding features:(160000, 34)
Index(['PostId', 'UserId', 'Comment', 'Body', 'Title', 'Tags', 'Reputation',
       'experienceInTime', 'WebsiteUrl', 'Location', 'AboutMe', 'Views',
       'Upvotes', 'Downvotes', 'ProfileImageUrl', 'tagCount', 'closed',
       'lowerUpperRatio', 'sentenceCount', 'firstLineLength', 'codeLength',
       'urlCount', 'SOUrlCount', 'titleLengthInChars',
       'countOfInterrogativeSent', 'sentencesStartWithYouCount',
       'sentencesStartWithICount', 'shortWordCount', 'bodyWordCount',
       'punctuationCount', 'websiteUrlFilled', 'locationFilled',
       'aboutMeFilled', 'profileImageUrlFilled'],
      dtype='object')


In [43]:
#Exporting dataframe after adding columns, Now: 34 columns
openCloseData.to_csv('openCloseData.csv')

In [ ]:
#https://www.ritchieng.com/pandas-scikit-learn/

print(openCloseData.shape)
openCloseData.dropna(subset=['Reputation', 'experienceInTime', 'Views', 'Upvotes', 'Downvotes',
       'lowerUpperRatio', 'sentenceCount',
       'firstLineLength', 'codeLength', 'urlCount', 'SOUrlCount',
       'titleLengthInChars', 'countOfInterrogativeSent',
       'sentencesStartWithYouCount', 'sentencesStartWithICount',
       'shortWordCount', 'bodyWordCount', 'tagCount', 'punctuationCount',
       'websiteUrlFilled', 'locationFilled', 'aboutMeFilled',
       'profileImageUrlFilled'],inplace = True,axis=0) 
print(openCloseData.shape)

X = openCloseData.loc[:, ['Reputation', 'experienceInTime', 'Views', 'Upvotes', 'Downvotes',
       'lowerUpperRatio', 'sentenceCount',
       'firstLineLength', 'codeLength', 'urlCount', 'SOUrlCount',
       'titleLengthInChars', 'countOfInterrogativeSent',
       'sentencesStartWithYouCount', 'sentencesStartWithICount',
       'shortWordCount', 'bodyWordCount', 'tagCount', 'punctuationCount',
       'websiteUrlFilled', 'locationFilled', 'aboutMeFilled',
       'profileImageUrlFilled']]
y = openCloseData.closed
print(type(X))
print(X.shape)
print(y.shape)
print(y)

In [ ]:
X.isnull()

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression()
logreg.fit(X,y)

In [ ]:
#TRAIN/CV/TEST SPLIT


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

bins = np.linspace(0,100,20)
#plt.hist(openCloseData[openCloseData['closed']==1]['Reputation'],bins,normed=True)
plt.hist(openCloseData[openCloseData['closed']==1]['titleLengthInChars'],bins,normed=True,label='Title_length')
#plt.hist(openCloseData['bodyWordCount'],bins,label='BodyWordCount')
#plt.xlabel('')
#plt.ylabel("Number of users")

#plt.legend(loc='upper right')
plt.show()

In [ ]:
#plt.hist(openCloseData[openCloseData['closed']==1]['Reputation'],bins=np.linspace(0,400,40),normed=True,label='Reputation')

#plt.show()

In [ ]:
X.drop(['lowerUpperRatio'], axis = 1, inplace = True)
X.dtypes
y.dtypes

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

#np.nan_to_num(X['lowerUpperRatio'])

X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2)
rf = RandomForestClassifier(n_estimators=50, max_depth = 20 , n_jobs = -1)
rf_model = rf.fit(X_train,y_train)
print(X_train)
print(y_train)
print(y_test)
#k_fold = KFold(n_splits=5)
    
#cross_val_score(rf,)

In [ ]:
sorted(zip(rf_model.feature_importances_,X_train.columns),reverse=True)[0:10]

In [ ]:
y_pred = rf_model.predict(X_test)

precision,recall,fscore,support = score(y_test,y_pred,pos_label=1,average='binary')
print("Precision : {} / Recall : {} / Accuracy : {} / ".format(round(precision,3),
                                                               round(recall,3),
                                                               round((y_pred==y_test).sum()/len(y_pred),3)))

In [ ]:
print(len(y_pred))

for a,b in zip(y_pred,y_test):
    print(str(a)+"=="+str(b))

In [ ]:
from sklearn.model_selection import KFold,cross_val_score
k_fold = KFold(n_splits=5)
    
cross_val_score(rf,X,y,cv=k_fold,scoring='accuracy',n_jobs=-1)

## CLASSIFIER FOR REASON FOR CLOSURE